## 02_silver_transformations
### Purpose: build Silver layer (clean, normalize, deduplicate)
Inputs: *__raw_

Outputs: *__clean_

### Clients table

In [0]:
CREATE OR REPLACE VIEW curso.ventas.clientes_clean AS
WITH 
base AS (
  SELECT
    id_cliente,

    -- Strings
    NULLIF(TRIM(nombre), '') AS nombre,
    NULLIF(TRIM(apellido), '') AS apellido,

    -- Normalización de sexo
    CASE
      WHEN LOWER(TRIM(sexo)) IN ('m', 'masculino') THEN 'M'
      WHEN LOWER(TRIM(sexo)) IN ('f', 'femenino')  THEN 'F'
      WHEN LOWER(TRIM(sexo)) IN ('otro') THEN 'O'
      ELSE NULL
    END AS sexo,

    -- Fecha nacimiento
    fecha_nacimiento,

    CURRENT_TIMESTAMP() AS cleaned_at
  FROM curso.ventas.clientes_raw
),

dedup AS (
  SELECT *
  FROM base
  WHERE id_cliente IS NOT NULL
  QUALIFY ROW_NUMBER() OVER (
    PARTITION BY id_cliente
    ORDER BY cleaned_at DESC
  ) = 1
)

SELECT id_cliente,
       nombre,
       apellido,
       sexo,
       fecha_nacimiento
FROM dedup;


In [0]:
-- The view "curso.ventas.clientes_clean" must be created before querying it.
-- Run the CREATE OR REPLACE VIEW statement from Cell 2 first, then execute:

SELECT *
FROM curso.ventas.clientes_clean
LIMIT 10;

id_cliente,nombre,apellido,sexo,fecha_nacimiento
1,Irma,Agudo,F,1954-04-27
2,Calixto,Sosa,F,1958-12-27
3,Dionisia,Bonet,null,1977-04-13
4,Lupe,Martorell,M,1991-04-22
5,Ágata,Valls,M,1968-08-19
6,Cesar,Montes,null,1951-07-07
7,Eli,Gras,null,2002-11-16
8,Marcos,Canet,null,1973-06-09
9,Verónica,Abad,O,1998-02-04
10,Florencia,Arenas,M,1988-09-05


### Products table

In [0]:
CREATE OR REPLACE VIEW curso.ventas.productos_clean AS
WITH
base AS(
  SELECT
    id_producto,

    -- Strings
    NULLIF(REGEXP_REPLACE(TRIM(familia), '(?i)^null$', ''), '') AS familia,
    NULLIF(REGEXP_REPLACE(TRIM(nombre), '(?i)^null$', ''), '') AS nombre,

    precio_unitario,

    CURRENT_TIMESTAMP() AS cleaned_at
  FROM curso.ventas.productos_raw
),

dedup AS (
  SELECT *
  FROM base
  WHERE id_producto IS NOT NULL
  QUALIFY ROW_NUMBER() OVER(
    PARTITION BY id_producto
    ORDER BY cleaned_at DESC
  ) = 1
)

SELECT id_producto,
       familia,
       nombre,
       precio_unitario
FROM dedup;

In [0]:
-- The view "curso.ventas.productos_clean" must be created before querying it.
-- Run the CREATE OR REPLACE VIEW statement from Cell 6 first, then execute:

SELECT * 
FROM curso.ventas.productos_clean
LIMIT 10;

id_producto,familia,nombre,precio_unitario
1,Leche,Leche,62
2,Leche,Nido,177
3,Leche,Klim,36
4,Leche,Nan Pro,156
5,Leche,Nestogeno,134
6,Leche,La lechera,68
7,Leche,Canprolac,36
8,Leche,"Sativa 2,3",22
9,Leche,Nidina,247
10,Leche,Nan H.a.,149


### Employees table

In [0]:
CREATE OR REPLACE VIEW curso.ventas.empleados_clean AS
WITH
base AS(
  SELECT
    id_vendedor,

    sucursal,

    -- Strings
    NULLIF(REGEXP_REPLACE(TRIM(nombre), '(?i)^null$', ''), '') AS nombre,
    NULLIF(REGEXP_REPLACE(TRIM(apellido), '(?i)^null$', ''), '') AS apellido,

    CURRENT_TIMESTAMP() AS cleaned_at
  
    FROM curso.ventas.empleados_raw
),
dedup AS(
  SELECT *
  FROM base
  WHERE id_vendedor IS NOT NULL
  QUALIFY ROW_NUMBER() OVER(
    PARTITION BY id_vendedor
    ORDER BY cleaned_at DESC
  ) = 1
)
SELECT id_vendedor,
       sucursal,
       nombre,
       apellido
FROM dedup;


In [0]:
-- The view "curso.ventas.empleados_clean" must be created before querying it.
-- Run the CREATE OR REPLACE VIEW statement from Cell 9 first, then execute:

SELECT id_vendedor,
       sucursal,
       nombre,
       apellido
FROM curso.ventas.empleados_clean
LIMIT 10;

id_vendedor,sucursal,nombre,apellido
1,14,Maria,Rodríguez
2,1,Juan,Gómez
3,4,Jose,González
4,1,Carlos,Martínez
5,1,Jorge,García
6,6,Luis,Fernández
7,7,Miguel,Rodríguez
8,4,Ana,González
9,8,Hector,null
10,7,Ramon,López


### Shops table

In [0]:
CREATE OR REPLACE VIEW curso.ventas.locales_clean AS
WITH base AS(
  SELECT
    id_sucursal,

    --Strings
    NULLIF(REGEXP_REPLACE(TRIM(nombre), '(?i)^null$', ''), '') as nombre,
    NULLIF(REGEXP_REPLACE(TRIM(tipo), '(?i)^null$', ''), '') as tipo,

    CURRENT_TIMESTAMP() AS cleaned_at
    
    FROM curso.ventas.locales_raw
),
dedup AS(
  SELECT * 
  FROM base
  WHERE id_sucursal IS NOT NULL
  QUALIFY ROW_NUMBER() OVER(
    PARTITION BY id_sucursal
    ORDER BY cleaned_at DESC
  ) = 1
)
SELECT id_sucursal,
       nombre,
       tipo
FROM dedup;

In [0]:
-- The view "curso.ventas.locales_clean" must be created before querying it.
-- Run the CREATE OR REPLACE VIEW statement from Cell 12 first, then execute:

SELECT * 
FROM curso.ventas.locales_clean
LIMIT 10;

id_sucursal,nombre,tipo
1,Éxito Aranjuez,Supermercado
2,Éxito Aventura,Comercio de cercania
3,Éxito Belén,Supermercado
4,Éxito Colombia,Hipermercado
5,Éxito Del Este,Vecino
6,Éxito Envigado,Hipermercado
7,Éxito Envigado Centro,Supermercado
8,Éxito Gran Vía,Supermercado
9,Éxito Indiana Mall,Comercio de cercania
10,Éxito La Ceja,Comercio de cercania


### Sales table (fact)

In [0]:
CREATE OR REPLACE VIEW curso.ventas.facturas_clean AS
WITH base AS(
  SELECT
    -- String
    NULLIF(REGEXP_REPLACE(TRIM(num_factura), '(?i)^null$', ''), '') AS num_factura,
    
    fecha_venta,
    producto as id_producto, -- Aligning the column names with the other tables
    vendedor as id_vendedor,
    cliente as id_cliente,
    cantidad,

    CURRENT_TIMESTAMP() AS cleaned_at

    FROM curso.ventas.facturas_raw
),
dedup AS(
  SELECT *
  FROM base
  WHERE num_factura IS NOT NULL
  QUALIFY ROW_NUMBER() OVER(
    PARTITION BY num_factura
    ORDER BY cleaned_at DESC
  ) = 1
)
SELECT num_factura,
       fecha_venta,
       id_producto,
       id_vendedor,
       id_cliente,
       cantidad
FROM dedup;

In [0]:
-- The view "curso.ventas.facturas_clean" must be created before querying it.
-- Run the CREATE OR REPLACE VIEW statement from Cell 15 first, then execute:

SELECT * 
FROM curso.ventas.facturas_clean
LIMIT 10;

num_factura,fecha_venta,id_producto,id_vendedor,id_cliente,cantidad
0000-05444682,2022-11-13,86,38,1668,56
0000-11294112,2022-08-10,90,63,2447,84
0000-13526698,2021-08-22,106,33,1109,66
0000-32356836,2022-10-13,115,21,1509,156
0000-52191864,2022-06-20,112,15,1042,155
0000-60937597,2022-03-25,109,24,1016,198
0000-63017602,2023-08-14,60,14,1882,42
0000-83897840,2022-03-28,88,58,468,41
0000-85450647,2022-08-09,14,33,181,114
0000-86250165,2023-02-16,38,11,306,29
